# CNN MODEL DEFINITION AND TRAINING

In [1]:
# Setup

# from keras.layers import Dense, Conv1D, Flatten, Activation, Lambda, GlobalAveragePooling1D
# from keras import regularizers
# import numpy as np
# import tensorflow as tf
# import os
from keras.models import Model
from keras.layers import Input, Conv1D, Dense, Activation, Dot, Flatten, Reshape, MaxPooling1D, Dropout
from keras.optimizers import Adam
from keras.backend import l2_normalize
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
from keras.callbacks import EarlyStopping
import typing

import random
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from datagenerator import DataGenerator

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len (physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

data_folder = os.getcwd() + '/data/eeg'
all_files = [f for f in os.listdir(data_folder) if f.endswith('.npz')]
train_generator = DataGenerator(all_files, batch_size=10, stage='train')
val_generator = DataGenerator(all_files, batch_size=10, stage='val')

# BUILDING THE MODEL
time_window = train_generator.time_window
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

eeg = Input(shape=(time_window, 64))
env1 = Input(shape=(time_window, 1))
env2 = Input(shape=(time_window, 1))

conv1 = Conv1D(filters=16, kernel_size=16, activation='relu', padding='causal')(eeg)

cos_sim1 = tf.keras.layers.Dot(axes=(1, 1), normalize=True)([conv1, env1])
cos_sim2 = tf.keras.layers.Dot(axes=(1, 1), normalize=True)([conv1, env2])

# Classification
out1 = Dense(1, activation="sigmoid")(tf.keras.layers.Flatten()(tf.keras.layers.Concatenate()([cos_sim1, cos_sim2])))

# 1 output per batch
out = tf.keras.layers.Reshape([1], name="output_name")(out1)
model = tf.keras.Model(inputs=[eeg, env1, env2], outputs=[out])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
# Fit the model
history = model.fit(train_generator, epochs=50, verbose=2, validation_data=val_generator, callbacks=[early_stopping])

training_loss = history.history['loss']
training_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracty = history.history['val_accuracy']

# Print the training and validation loss and accuracy
print("Training Loss:", training_loss)
print("Training Accuracy:", training_accuracy)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].plot(training_loss, label='Training Loss')
axs[0].plot(val_loss, label='Validation Loss')
axs[0].legend()

axs[1].plot(training_accuracy, label='Training Accuracy')
axs[1].plot(val_accuracty, label='Validation Accuracy')
axs[1].legend()

plt.show()

Num GPUs Available:  1
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-04-19 21:25:12.217795: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-19 21:25:12.218949: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2023-04-19 21:25:13.756898: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-19 21:25:14.097568: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-19 21:25:41.046501: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


31/31 - 32s - loss: 0.6901 - accuracy: 0.5329 - val_loss: 0.6897 - val_accuracy: 0.5264 - 32s/epoch - 1s/step
Epoch 2/50
31/31 - 29s - loss: 0.6853 - accuracy: 0.5553 - val_loss: 0.6871 - val_accuracy: 0.5484 - 29s/epoch - 946ms/step
Epoch 3/50


KeyboardInterrupt: 

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 640, 64)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 640, 16)      16400       ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 640, 1)]     0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 640, 1)]     0           []                               
                                                                                              

In [3]:
test_generator = DataGenerator(all_files, batch_size=10, stage='test')

# Test the model
for i in range(len(all_files) - 2):
    (inputs), labels = test_generator.__getitem__(i)

    predictions = model.predict(inputs)
    correct = 0
    for prediction, label in zip(predictions, labels):
        if prediction > 0.5 and label == 1:
            correct += 1
        elif prediction < 0.5 and label == 0:
            correct += 1
            
    print("Test Accuracy:", correct / len(predictions))

2023-04-19 16:48:25.644941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 103ms/step
Test Accuracy: 0.5266666666666666
6/6 [==============================] - 0s 76ms/step
Test Accuracy: 0.6829268292682927
5/5 [==============================] - 0s 78ms/step
Test Accuracy: 0.5362318840579711
6/6 [==============================] - 0s 53ms/step
Test Accuracy: 0.5333333333333333
6/6 [==============================] - 0s 47ms/step
Test Accuracy: 0.6111111111111112
6/6 [==============================] - 0s 65ms/step
Test Accuracy: 0.5602409638554217
6/6 [==============================] - 0s 22ms/step
Test Accuracy: 0.5304878048780488
6/6 [==============================] - 0s 55ms/step
Test Accuracy: 0.5393258426966292
6/6 [==============================] - 0s 67ms/step
Test Accuracy: 0.5454545454545454
5/5 [==============================] - 0s 56ms/step
Test Accuracy: 0.65
5/5 [==============================] - 0s 4ms/step
Test Accuracy: 0.50625
5/5 [==============================] - 0s 4ms/step
Test Accuracy: 0.6875
5/5 [=

AttributeError: 'NoneType' object has no attribute 'reshape'